In [3]:
import sys
!{sys.executable} -m pip install pandas

In [8]:
pd.set_option("display.max_columns", None)

In [34]:
import pandas as pd
import os
import datetime
import time
import random

In [119]:
data = {}
param = {}
input_directory = '../temp'
param_directory = '../param'
for _filename in os.listdir(input_directory):
    if _filename[0] == '.':
        continue
    print(_filename)
    df_data = pd.read_csv(input_directory+'/'+_filename,  header = None)
    filetype = _filename.split('-')[1]
    column_dict = { 
            'sked': ['from', 'to', 'al','fln', 'actype', 'depday','dep','arrday','arr'],
            'comp': ['from', 'to', 'al', 'fln','actype', 'depday','dep','arrday','arr'],
            'demand': ['orig', 'dest', 'volume','weight', 'rev'],
            'demand_curve': ['orig', 'dest', 'ttt_1','ttt_2', 'ttt_3','ttt_4'],          
            'route_cost': ['from','to', 'actype', 'value'],
            'airport_cost': ['ap', 'actype', 'value'],
            'config': ['actype', 'cap_1', 'cap_2']
    }
    df_data.columns = column_dict[filetype]
    data[filetype] = df_data
    
for _filename in os.listdir(param_directory):
    if _filename[0] == '.':
        continue
    print(_filename)
    df_data = pd.read_csv(param_directory+'/'+_filename,  header = None)
    column_dict = { 
            'network' : ['stops_allowed'],
            'connections': ['hub', 'minct', 'maxct'],
            'preferences' : ['from', 'to', 'nonstop', 'connect']
    }
    df_data.columns = column_dict[_filename]
    param[_filename] = df_data

test-sked
test-demand_curve
test-comp
test-demand
test-route_cost
test-config
test-airport_cost
preferences
connections


In [123]:
your_sked = data['sked']
comp_sked = data['comp']
param_connect = param['connections']

full_sked = your_sked.append(comp_sked)
full_sked["dep"] = pd.to_datetime(full_sked["dep"]) + full_sked["depday"]*datetime.timedelta(days=1)
full_sked["arr"] = pd.to_datetime(full_sked["arr"]) + full_sked["arrday"]*datetime.timedelta(days=1)
full_sked["id"] = full_sked['al'] + full_sked['fln'].astype('str')


param_connect['minct'] = pd.to_timedelta(param_connect['minct'])
param_connect['maxct'] = pd.to_timedelta(param_connect['maxct'])

df_connect = full_sked.add_suffix('_1').reset_index(drop=True)
df_connect['nbstops'] = 0

list_itin = {}
list_itin[0] = df_connect

#build itineraries
for i in range (1,3):
    cnx_p = param_connect.copy()
    itins = list_itin[i-1].merge(full_sked.add_suffix('_'+str(i+1)), how= "cross")
    itins['nbstops'] = i
    
    itins['cnx_time_'+str(i)] = itins['dep_'+str(i+1)] - itins['arr_'+str(i)]
    
    cnx_p.columns = ['to_'+str(i), 'minct_'+str(i), 'maxct_'+str(i)]
    
    #filter connections
    itins.drop(itins[itins['to_'+str(i)] != itins['from_'+str(i+1)]].index, inplace=True)
    itins.drop(itins[itins['from_'+str(i)] == itins['to_'+str(i+1)]].index, inplace=True)
    itins.drop(itins[itins['from_'+str(1)] == itins['to_'+str(i+1)]].index, inplace=True)
    
    itins = itins.merge(cnx_p, on = ['to_'+str(i)])
    
    itins.drop(itins[itins['cnx_time_'+str(i)] < itins['minct_'+str(i)]].index, inplace=True)
    itins.drop(itins[itins['cnx_time_'+str(i)] > itins['maxct_'+str(i)]].index, inplace=True)
   
    list_itin[i] = itins

#add attributes
for i in range(0,3):
    list_itin[i]["travel_time"]=list_itin[i]["arr_"+str(i+1)]-list_itin[i]["dep_1"]
    list_itin[i]["od"]=list_itin[i]["from_1"]+list_itin[i]["to_"+str(i+1)]


In [121]:
#define user arrivals
demand_rand = {}
demand_curve = data['demand_curve']
demand = data['demand']
demand_by_ttt = pd.merge(demand, demand_curve, on=["orig","dest"])

for i in range(1,5):
    demand_by_ttt['d_ttt_'+str(i)]=(demand_by_ttt['ttt_'+str(i)]*demand_by_ttt['volume']).astype('int')

#shuffle pax
randm = []
for i in range(1,5):
    for indx, row in demand_by_ttt.iterrows():
        for k in range(1,row['d_ttt_'+str(i)]):
            randm.append(row['orig']+row['dest'])
    random.shuffle(randm)
    demand_rand[i] = randm
    

In [111]:
demand_rand

{1: ['DUSFRA',
  'DUSLHR',
  'DUSCDG',
  'DUSFCO',
  'DUSTXL',
  'DUSBCN',
  'DUSTXL',
  'FRAMAD',
  'DUSMAD',
  'DUSMAD',
  'DUSFCO',
  'DUSLHR',
  'DUSLHR',
  'DUSFCO',
  'DUSCDG',
  'DUSFRA',
  'DUSFRA',
  'LHRMAD',
  'DUSCDG',
  'DUSTXL',
  'DUSBCN',
  'DUSLHR',
  'DUSCDG',
  'TXLMAD',
  'DUSTXL',
  'DUSBCN',
  'TXLMAD',
  'DUSCDG',
  'FCOMAD',
  'DUSMAD',
  'DUSFRA',
  'DUSLHR',
  'DUSFRA',
  'DUSFRA',
  'DUSCDG',
  'DUSCDG',
  'DUSMAD',
  'DUSBCN',
  'DUSFRA',
  'DUSFCO',
  'FRAMAD',
  'DUSMAD',
  'DUSTXL',
  'DUSFRA',
  'DUSFRA',
  'DUSFCO',
  'DUSTXL',
  'DUSCDG',
  'BCNMAD',
  'DUSMAD',
  'TXLMAD',
  'DUSTXL',
  'DUSTXL',
  'FRAMAD',
  'DUSCDG',
  'DUSBCN',
  'CDGMAD',
  'DUSLHR',
  'DUSFRA',
  'DUSLHR',
  'DUSMAD',
  'DUSFRA',
  'DUSFRA',
  'DUSCDG',
  'DUSMAD',
  'DUSMAD',
  'FRAMAD',
  'CDGMAD',
  'DUSCDG',
  'DUSFRA',
  'CDGMAD',
  'DUSBCN',
  'DUSFCO',
  'DUSFRA',
  'DUSFRA',
  'DUSFCO',
  'DUSCDG',
  'DUSBCN',
  'DUSFCO',
  'DUSBCN',
  'DUSLHR',
  'BCNMAD',
  'DUSBCN',
 

In [124]:
list_itin[1].head()

,from_1,to_1,al_1,fln_1,actype_1,depday_1,dep_1,arrday_1,arr_1,id_1,...,depday_2,dep_2,arrday_2,arr_2,id_2,cnx_time_1,minct_1,maxct_1,travel_time,od
0,DUS,CDG,FF,2,737,1,2022-01-24 14:14:00,1,2022-01-24 17:00:00,FF2,...,1,2022-01-24 18:14:00,1,2022-01-24 19:15:00,AF21,0 days 01:14:00,0 days 00:15:00,0 days 08:00:00,0 days 05:01:00,DUSTXL
7,DUS,CDG,LH,2,737,1,2022-01-24 14:00:00,1,2022-01-24 17:00:00,LH2,...,1,2022-01-24 18:14:00,1,2022-01-24 19:15:00,AF21,0 days 01:14:00,0 days 00:15:00,0 days 08:00:00,0 days 05:15:00,DUSTXL
16,TXL,CDG,AF,15,380,1,2022-01-24 10:19:00,1,2022-01-24 12:41:00,AF15,...,1,2022-01-24 16:16:00,1,2022-01-24 18:37:00,AF22,0 days 03:35:00,0 days 00:15:00,0 days 08:00:00,0 days 08:18:00,TXLFRA
17,TXL,CDG,AF,15,380,1,2022-01-24 10:19:00,1,2022-01-24 12:41:00,AF15,...,1,2022-01-24 15:03:00,1,2022-01-24 16:32:00,AF23,0 days 02:22:00,0 days 00:15:00,0 days 08:00:00,0 days 06:13:00,TXLLHR
18,TXL,CDG,AF,15,380,1,2022-01-24 10:19:00,1,2022-01-24 12:41:00,AF15,...,1,2022-01-24 16:53:00,1,2022-01-24 17:34:00,AF24,0 days 04:12:00,0 days 00:15:00,0 days 08:00:00,0 days 07:15:00,TXLMAD


In [37]:
array_ = list(range(1,1000000))

In [39]:
start = time.time()
random.shuffle(array_)
end = time.time()
delta = end - start

In [40]:
delta

1.3008878231048584

In [ ]:
#def connection_builder(your_sked, comp_sked, param_connect):
    your_sked = data['sked']
    comp_sked = data['comp']
    param_connect = param['connections']


    #create df from data
    full_sked = your_sked.append(comp_sked)
    df_connect["dep"] = pd.to_datetime(df_connect["dep"]) + df_connect["depday"]*datetime.timedelta(days=1)
    df_connect["arr"] = pd.to_datetime(df_connect["arr"]) + df_connect["arrday"]*datetime.timedelta(days=1)
    
    df_connect = your_sked.copy()
    df_connect = df_connect.append(comp_sked)
    df2ndleg = your_sked.copy()
    df2ndleg = df2ndleg.append(comp_sked) 
    
    df_connect = df_connect.merge(df2ndleg, how='cross')
    df_param = param_connect

    #convert formats and create new fields
    df_connect["dep_x"] = pd.to_datetime(df_connect["dep_x"]) + df_connect["day_x"]*datetime.timedelta(days=1)
    df_connect["arr_x"] = pd.to_datetime(df_connect["arr_x"]) + df_connect["day_x"]*datetime.timedelta(days=1)
    df_connect["dep_y"] = pd.to_datetime(df_connect["dep_y"]) + df_connect["day_y"]*datetime.timedelta(days=1)
    df_connect["arr_y"] = pd.to_datetime(df_connect["arr_y"]) + df_connect["day_y"]*datetime.timedelta(days=1)
    df_param['minct'] = pd.to_timedelta(df_param['minct'])
    df_param['maxct'] = pd.to_timedelta(df_param['maxct'])

    df2ndleg.columns = ['from_x', 'to_x', 'al_x', 'actype_x', 'day_x','dep_x','arr_x']
    df2ndleg["dep_x"] = pd.to_datetime(df2ndleg["dep_x"]) + df2ndleg["day_x"]*datetime.timedelta(days=1)
    df2ndleg["arr_x"] = pd.to_datetime(df2ndleg["arr_x"]) + df2ndleg["day_x"]*datetime.timedelta(days=1)
    df2ndleg["stops"] = 0

    df_connect['cnx_time'] = df_connect.dep_y - df_connect.arr_x
    df_connect['stops'] = 1

    #filter connections
    df_connect = df_connect.merge(df_param,on=['to_x'] ).copy()
    df_connect = df_connect[df_connect.to_x == df_connect.from_y].copy()
    df_connect = df_connect[df_connect.from_x != df_connect.to_y].copy()
    df_connect = df_connect[df_connect.cnx_time >= df_connect.minct].copy()
    df_connect = df_connect[df_connect.cnx_time <= df_connect.maxct].copy()


    #create df with all itineraries
    df_itin = df_connect.append(df2ndleg).reset_index()

    #add fields for itinerary quality
    df_itin["traveltime"] = df_itin.arr_x - df_itin.dep_x
    df_itin.loc[df_itin['stops']==1, ["traveltime"]] = df_itin.arr_y - df_itin.dep_x
    df_itin["traveltime"]= pd.to_timedelta(df_itin["traveltime"].copy())
    
#    return df_itin